In [ ]:
#Script plotting Figure 24

import numpy as np
import matplotlib.pyplot as plt
import random

# Define the extinction model components
def drude(x, x0, gamma):
    return x**2 / ((x**2 - x0**2)**2 + x**2 * gamma**2)

def FUV_rise(x):
    return np.where(x >= 5.9, 0.5392 * (x - 5.9)**2 + 0.05644 * (x - 5.9)**3, 0)

def extinction_model(x, c1, c2, c3, c4, x0, gamma):
    return c1 + c2 * x + c3 * drude(x, x0, gamma) + c4 * FUV_rise(x)

# Sample 3 random objects from the dataset
fm_file = "Valencic Galaxy data_easyread.txt"
with open(fm_file, 'r') as f:
    lines = f.readlines()

data_lines = [line for line in lines if line.strip() and line[:14].strip()]
parsed_data = []
for line in data_lines:
    try:
        parsed_data.append([
            float(line[18:24]),  # c1_norm
            float(line[38:45]),  # c2_norm
            float(line[59:65]),  # c3_norm
            float(line[79:85]),  # c4_norm
            float(line[99:105]), # x0
            float(line[119:125]) # gamma
        ])
    except ValueError:
        continue

df_fm = pd.DataFrame(parsed_data, columns=['c1_norm', 'c2_norm', 'c3_norm', 'c4_norm', 'x0', 'gamma'])

selected_objects = random.sample(df_fm, 3)

# Wavenumber domain
x = np.linspace(3.0, 8, 1000)

# Plot
plt.figure(figsize=(10, 6))
for name, AV, x0_angstrom, gamma, c1, c2, c3, c4 in selected_objects:
    x0 = 1e4 / x0_angstrom  # convert Angstrom to μm⁻¹
    A_lambda_over_AV = extinction_model(x, c1, c2, c3, c4, x0, gamma)
    plt.plot(x, A_lambda_over_AV, label=name)

plt.xlabel(r'Wavenumber $x$ ($\mu$m$^{-1}$)', fontsize=12)
plt.ylabel(r'$A_\lambda / A_V$', fontsize=12)
plt.title('Extinction Curves from IUE MW data (3 Random Sight Lines)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("3_random_IUE_sight_lines_4.png", dpi=300)
plt.show()


In [ ]:
#Script to plot Figure 25, 27,28


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Load FM Data ----
fm_file = "Valencic Galaxy data_easyread.txt"
with open(fm_file, 'r') as f:
    lines = f.readlines()

data_lines = [line for line in lines if line.strip() and line[:14].strip()]
parsed_data = []
for line in data_lines:
    try:
        parsed_data.append([
            float(line[18:24]),  # c1_norm
            float(line[38:45]),  # c2_norm
            float(line[59:65]),  # c3_norm
            float(line[79:85]),  # c4_norm
            float(line[99:105]), # x0
            float(line[119:125]) # gamma
        ])
    except ValueError:
        continue

df_fm = pd.DataFrame(parsed_data, columns=['c1_norm', 'c2_norm', 'c3_norm', 'c4_norm', 'x0', 'gamma'])

# Load R(V) values
rv_file = "Valencic Galaxy data 2_easyread.txt"
with open(rv_file, 'r') as f:
    rv_lines = f.readlines()

rv_data_lines = [line for line in rv_lines if line.strip()]
rv_values = []
for line in rv_data_lines:
    try:
        rv = float(line[33:37])
        rv_values.append(rv)
    except ValueError:
        continue

# ---- Compute mean FM parameters ----
mean_fm = df_fm.mean()
avg_RV = np.mean(rv_values)
# ---- Compute standard deviations for error band ----
std_fm = df_fm.std()
std_c3 = std_fm['c3_norm']
std_x0 = std_fm['x0']
std_gamma = std_fm['gamma']

# ---- Compute upper and lower FM Drude curves ----
fm_upper = (mean_fm['c3_norm'] + std_c3) * drude(x, mean_fm['x0'], mean_fm['gamma']) / avg_RV
fm_lower = (mean_fm['c3_norm'] - std_c3) * drude(x, mean_fm['x0'], mean_fm['gamma']) / avg_RV



# ---- Define wavelength domain ----
x = np.linspace(3, 8, 1000)

# ---- Drude function ----
def drude(x, x0, gamma):
    return x**2 / ((x**2 - x0**2)**2 + (x * gamma)**2)

# ---- FM Drude bump ----
fm_drude = mean_fm['c3_norm'] * drude(x, mean_fm['x0'], mean_fm['gamma']) / avg_RV

# ---- Nanopyroxene Drude bump Taken from Table S1 ----
a1_nano = 4.474625874136247  # Mb
x01_nano = 4.418409864496625
gamma1_nano = 0.7457880706235912
a2_nano = 0.3202926367114822  # Mb
x02_nano = 4.650196514336919
gamma2_nano = 0.40882631138585734
nano_drude = a1_nano * drude(x, x01_nano, gamma1_nano) + a2_nano * drude(x, x02_nano, gamma2_nano)

# ---- PAH Drude bump (from Lin et al. 2023) ----
a1_pah = 21.2  # Mb/C
x01_pah = 4.40
gamma1_pah = 1.56
pah_drude = a1_pah * drude(x, x01_pah, gamma1_pah)

# ---- Normalize all to nanopyroxene peak ----
max_nano = np.max(nano_drude)
fm_drude_norm = fm_drude * (max_nano / np.max(fm_drude))
pah_drude_norm = pah_drude * (max_nano / np.max(pah_drude))
fm_upper_norm = fm_upper * (max_nano / np.max(fm_drude))
fm_lower_norm = fm_lower * (max_nano / np.max(fm_drude))

# ---- Plot with eV shift----
plt.figure(figsize=(10, 6))
plt.plot(x +.081, nano_drude,':', label='Nanopyroxene Drude Bump (Normalised', color='darkgreen')
plt.plot(x+.162, pah_drude_norm, '--', label='PAH Drude Bump (Normalised)', color='red')
plt.plot(x, fm_drude_norm, label='IUE Averaged Drude Bump (Normalised)', color='blue')
plt.fill_between(x, fm_lower_norm, fm_upper_norm, color='blue', alpha=0.1, label='±1σ Range')


plt.xlabel('Wavenumber x (μm$^{-1}$)', fontsize=12)
plt.ylabel('Absorption (Normalised (A.U))', fontsize=12)
plt.title('Comparison of Drude Bumps: Nanopyroxene, PAH, and IUE Observed Average', fontsize=14)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("Comparison_of_3_Drude_bumps.png", dpi=300)
plt.show()


In [ ]:
#Script to plot Figure 26

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d

# ---- 1. DEFINE X DOMAIN & SHIFTS ----
x_fit     = np.linspace(0.0, 9.0, 5000)
shift_nano = 0.081
shift_pah  = 0.162
x_nano    = x_fit + shift_nano
x_pah     = x_fit + shift_pah

# ---- 2. COMPONENT FUNCTIONS ----
def drude(x, x0, gamma):
    return x**2 / ((x**2 - x0**2)**2 + (x * gamma)**2)

def fano(x, x0, gamma, q):
    num = (x - x0 + q * gamma/2)**2
    den = (x - x0)**2 + (gamma/2)**2
    return num/den

def drude_poly(x, a1, x01, g1, a2, x02, g2, c0, c1, c2):
    d1   = a1 * drude(x, x01, g1)
    d2   = a2 * drude(x, x02, g2)
    poly = (c0 + c1*x + c2*x**2)*1e-18
    return d1 + d2 + poly

# ---- 3. NANOPYROXENE & PAH SPECTRA ----
nano_mb = drude_poly(
    x_fit,
    4.474625874136247e-18, 4.418409864496625, 0.7457880706235912,
    0.3202926367114822e-18, 4.650196514336919, 0.40882631138585734,
    12.042324475316557, -5.910331306098436, 0.8593439269821448
) / 1e-18

pah_mb = (
    21.2e-18 * drude(x_fit, 4.40, 1.56) +
    1.90e-18 * fano(x_fit, 12.22, 7.34, 3.34)
) / 1e-18

# ---- 4. LOAD FM DATA (easyread files) ----
# FM bump parameters
fm_file = "Valencic Galaxy data_easyread.txt"
with open(fm_file, 'r') as f:
    lines = f.readlines()

data_lines  = [L for L in lines if L.strip() and L[:14].strip()]
parsed_data = []
for L in data_lines:
    try:
        parsed_data.append([
            float(L[18:24]),  # c1_norm
            float(L[38:45]),  # c2_norm
            float(L[59:65]),  # c3_norm
            float(L[79:85]),  # c4_norm
            float(L[99:105]), # x0
            float(L[119:125]) # gamma
        ])
    except ValueError:
        continue

df_fm    = pd.DataFrame(parsed_data, columns=['c1_norm','c2_norm','c3_norm','c4_norm','x0','gamma'])
mean_fm  = df_fm.mean()
std_c3   = df_fm['c3_norm'].std()

# R(V) values
rv_file     = "Valencic Galaxy data 2_easyread.txt"
with open(rv_file, 'r') as f:
    rv_lines = f.readlines()

rv_data_lines = [L for L in rv_lines if L.strip()]
rv_values     = []
for L in rv_data_lines:
    try:
        rv_values.append(float(L[33:37]))
    except ValueError:
        continue

avg_RV = np.mean(rv_values)

# ---- 5. COMPUTE OBSERVED EXTINCTION & ERRORS ----
x_ext = np.linspace(0.0, 9.0, 500)
Drude = lambda x,x0,g: x**2 / ((x**2 - x0**2)**2 + (x*g)**2)

k_x = (
      mean_fm['c1_norm']
    + mean_fm['c2_norm'] * x_ext
    + mean_fm['c3_norm'] * Drude(x_ext, mean_fm['x0'], mean_fm['gamma'])
    + mean_fm['c4_norm'] * (0.5392*(x_ext-5.9)**2 + 0.05644*(x_ext-5.9)**3) * (x_ext>5.9)
)
AAv = k_x/avg_RV + 1.0

# ±1σ on the bump only
delta_k3 = std_c3 * Drude(x_ext, mean_fm['x0'], mean_fm['gamma'])
delta_A  = delta_k3 / avg_RV

# interpolate full A(λ)
ext_interp = interp1d(x_ext, AAv,     bounds_error=False, fill_value="extrapolate")(x_fit)
err_interp = interp1d(x_ext, delta_A, bounds_error=False, fill_value=0.0)(x_fit)

# ---- 6. TWO-POINT NORMALISATION ----
def two_point_norm(x_arr, y_arr):
    y1   = np.interp(3.0, x_arr, y_arr)
    mask = (x_arr>=4.0) & (x_arr<=5.0)
    y2   = y_arr[mask].max()
    return (y_arr - y1)/(y2 - y1), (y2 - y1)

nano_norm, scale_nano = two_point_norm(x_nano, nano_mb)
pah_norm,  scale_pah  = two_point_norm(x_pah,  pah_mb)
ext_norm,  scale_ext  = two_point_norm(x_fit,  ext_interp)
err_norm = err_interp / scale_ext

# ---- 7. PLOT ALL TOGETHER ----
plt.figure(figsize=(10,6))
plt.plot(x_nano, nano_norm, '-',      label='Nanopyroxene (shifted & normalised)', color='darkgreen')
plt.plot(x_pah,  pah_norm,  '--',    label='PAH (shifted & normalised)',         color='red')
plt.plot(x_fit,  ext_norm,  '-',     label='MW Extinction (normalised)',        color='blue')

plt.fill_between(x_fit, ext_norm-err_norm, ext_norm+err_norm,
                 alpha=0.2, label='Extinction ±1σ')

plt.axvline(4.6, color='k', linestyle='--')
plt.xlim(3, 9)
plt.ylim(-0.1, 1.9)
plt.xlabel(r'Wavenumber ($\mu$m$^{-1}$)', fontsize=12)
plt.ylabel('Normalised Absorption (A.U.)',        fontsize=12)
plt.title('PAH, P3-8+2H and Observed MW Extinction', fontsize=14)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("normalised_extinction_with_rise.png", dpi=300)
plt.show()
